In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'
import numpy as np
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


import matplotlib.pyplot as plt

cuda:0


In [4]:
import numpy as np

# Define the joint state space for two units (4 states each)
joint_states = [(i, j) for i in range(4) for j in range(4)]
num_joint_states = len(joint_states)
# Sort the joint states based on the sum of values in each tuple
joint_states = sorted(joint_states, key=lambda x: sum(x))
# Example: Define a 4x4 joint transition matrix P((s1, s2) -> (s1', s2'))
P_joint = np.array([
    # Each row corresponds to a joint state, each column to the next joint state
    [0.4, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0],  # (0,0) -> various states
    [0.3, 0.3, 0.1, 0.1, 0.07, 0.05, 0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (0,1)
    [0.2, 0.2, 0.3, 0.1, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.03, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,2)
    [0.1, 0.1, 0.2, 0.4, 0.05, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0],  # (0,3)
    [0.3, 0.2, 0.1, 0.1, 0.2, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0],  # (1,0)
    [0.2, 0.2, 0.1, 0.1, 0.3, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0],  # (1,1)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.4, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.01, 0.0, 0.0],  # (1,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.2, 0.1, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0],  # (1,3)
    [0.3, 0.1, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01],  # (2,0)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.1, 0.4, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01],  # (2,1)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.05, 0.05, 0.3, 0.05, 0.02, 0.02, 0.01, 0.01],  # (2,2)
    [0.1, 0.1, 0.2, 0.3, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.1, 0.3, 0.05, 0.05, 0.02, 0.02],  # (2,3)
    [0.2, 0.2, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.3, 0.1, 0.05, 0.05],  # (3,0)
    [0.1, 0.1, 0.2, 0.2, 0.05, 0.1, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.3, 0.05, 0.1],  # (3,1)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.05, 0.1, 0.4, 0.2],  # (3,2)
    [0.05, 0.05, 0.1, 0.1, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.1, 0.1, 0.2, 0.3],  # (3,3)
])

# Set lower triangular elements to 0
for i in range(16):
    for j in range(i):
        P_joint[i, j] = 0
# Set lower triangular elements to 0
for i in range(16):
    for j in range(i+5,16):
        P_joint[i, j] = 0

P_joint=P_joint+np.eye(16,16)*0.8
# Normalize each row so the sum of probabilities equals 1
P_joint = P_joint / P_joint.sum(axis=1, keepdims=True)

# Check if the rows sum to 1
row_sums = P_joint.sum(axis=1)
print(row_sums)  # Should all be close to 1







P_joint



T_matrix=torch.tensor([[0.99, 0.01, 0.0,0.0],
        [0.00, 0.99, 0.01,0],
        [0.0, 0.00, 0.99,.01],
                      [0.0,0.0,0.0,1]], device=device) 

T_matrix1=torch.zeros(4,4,4)
T_matrix1[0,:,:]=T_matrix+torch.eye(4,4)*-0.0
T_matrix1[1,:,:]=T_matrix+torch.eye(4,4)*-0.3
T_matrix1[2,:,:]=T_matrix+torch.eye(4,4)*-0.5
T_matrix1[3,:,:]=T_matrix+torch.eye(4,4)*-0.7

T_matrix1=T_matrix1/torch.sum(T_matrix1, 2).reshape(4,4,1)

T_matrix1


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,3.5,5.5,7.5]).to(device)
O_matrix_std=torch.tensor([1.0,1.0,1.0,1.0]).to(device)


print(T_matrix)

print(s_0)



from torch.distributions.multivariate_normal import MultivariateNormal

def simulate_HMM(T_matrix_1,O_matrix,x,t):
    x=(0,0)
    current_state_idx = joint_states.index(x)
    states=[]
    t=0
    o=[]
    o1=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
    o.append(Normal(O_matrix_mean[x[0]],1).sample().to(device))
    o1.append(Normal(O_matrix_mean[x[1]],1).sample().to(device))
    t=1
    h=torch.tensor([1,0,0,0.0])
    states.append(x)
    for i in range(1,5000):
#         print(i)
        # print(states)
        t+=1
        p = P_joint[current_state_idx]
        # print(current_state_idx)
        # Choose the next joint state based on the current transition probabilities
        next_state_idx = np.random.choice(range(num_joint_states), p=p)
        next_state = joint_states[next_state_idx]
        x=next_state
        # x[0]=x1.item()
        # x[1]=x2.item()
        states.append(x)
        # print(x[0])
        # print(alpha)
        o_1,o_2 = MultivariateNormal(torch.stack([O_matrix_mean[x[0]], O_matrix_mean[x[1]]]), 
                                     torch.tensor([[1, 0.0],
                              [0.0, 1]])).sample().t().to(device)
        o.append(o_1)
        o1.append(o_2)
        current_state_idx=next_state_idx

        
        t1=t
        
        if x[0]==3 and x[1]==3:
            break
    
    
    
    o=torch.tensor(o).to(device)
    o1=torch.tensor(o1).to(device)
    return o,o1,t

o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
tensor([[0.9900, 0.0100, 0.0000, 0.0000],
        [0.0000, 0.9900, 0.0100, 0.0000],
        [0.0000, 0.0000, 0.9900, 0.0100],
        [0.0000, 0.0000, 0.0000, 1.0000]])
tensor([1., 0., 0.])


(tensor([1.0308, 2.6286, 0.2883, 2.0794, 0.4186, 2.3832, 0.7818, 1.7476, 0.7760,
         1.5367, 3.1158, 2.9485, 1.9880, 2.4939, 0.4020, 1.7783, 0.9133, 1.2790,
         0.9037, 2.4191, 4.9690, 7.4854, 5.7155, 5.2982, 6.4865, 6.9642, 5.9387,
         4.8929, 2.6011, 4.4252, 2.6974, 4.2001, 2.6792, 3.5535, 2.8258, 3.4850,
         4.8621, 4.0178, 2.3554, 4.1098, 4.9670, 3.4998, 3.3502, 2.3412, 2.0349,
         5.2763, 2.0069, 2.4146, 3.4179, 4.2388, 3.6676, 5.3228, 7.4585, 8.2261,
         7.7235, 7.5856, 9.1175, 6.7499, 9.2200, 4.7516, 7.1384, 7.1008, 7.2744,
         8.9772, 8.1734, 5.7904, 6.7842, 4.1113, 4.5409, 3.7802, 4.2437, 9.7552,
         7.9684]),
 tensor([2.6020, 1.2213, 1.0434, 1.8409, 0.6141, 1.9081, 6.0314, 5.4884, 5.1548,
         4.7756, 5.6239, 5.8536, 5.7667, 6.6545, 5.1310, 5.5100, 4.6097, 4.5356,
         5.1057, 6.5017, 0.8986, 1.9190, 1.2299, 2.0765, 2.1086, 2.8593, 3.6437,
         1.9360, 5.2476, 5.2424, 6.1105, 5.1751, 4.4084, 5.5583, 5.9256, 6.7690,
         

In [5]:
plt.plot(o[0].cpu())
# plt.plot(o[1])

In [6]:
plt.plot(o[0].cpu())
plt.plot(o[1].cpu())

In [7]:
# pred1=[]
# for i in range(0, 100):
#             pred1.append(simulate_HMM(T_matrix, O_matrix_mean, s_0, 1))

# torch.save(pred1, 'DATA_NON_LINEAR.pth')

# print("pred1 list saved successfully.")

In [8]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])

def var_diff(o_1):
    return torch.sum(o_1[1])

def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k



# MAIN MODEL 

In [9]:

class ATTENTION(nn.Module):
    def __init__(self, num_states=4):
        super(ATTENTION, self).__init__()
        num_heads=1
        hidden_dim=4
        self.num_states=4
        
        self.input_dim = num_states
        input_dim=num_states

        self.input_projection_1 = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.input_projection1_1 = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.input_projection2_1 = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.attention_1 = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        self.fc_1 = nn.Linear(hidden_dim ,hidden_dim)
        

    def forward(self,  Q, K, V, t1):

        seq_len = Q.size(1)
        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=Q.device), diagonal=1)
        # print(attn_mask)
        attn_mask = attn_mask.masked_fill(attn_mask == 1, float('-inf')).masked_fill(attn_mask == 0, float(0.0))

        
        projected_inputs = self.input_projection_1(Q)  # [batch, seq_len, hidden_dim]
        projected_inputs1 = self.input_projection1_1(K)  # [batch, seq_len, hidden_dim]
        projected_inputs2 = self.input_projection2_1(V)  # [batch, seq_len, hidden_dim]
        attn_output, _ = self.attention_1(projected_inputs, projected_inputs1, projected_inputs2, attn_mask=attn_mask)  # [batch, seq_len, hidden_dim]
        pred = (F.tanh(self.fc_1(attn_output))+(t1))   # [batch, seq_len, output_dim]

        return pred


class PositionalEncoding(nn.Module):  #@save
    """Positional encoding."""
    def __init__(self, num_hiddens, dropout, max_len=300):
        super().__init__()
        self.base =3*(torch.tensor([0.0,0.25,0.5,1]))
        self.dropout = nn.Dropout(dropout)
        # Create a long enough P
        self.P = torch.zeros((1, max_len, num_hiddens))
        X = 3*torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1)/max_len #/ torch.pow(10000, torch.arange(
            #0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        for i in range(4):
            self.P[:, :, i:i+1] = torch.cos(self.base[i]-X)
        
        # self.P[:, :, 0::2] = torch.cos(X)
        # self.P[:, :, 1::2] = torch.cos(X)

    def forward(self, X):
        X = X + self.P[:, :X.shape[1], :].to(X.device)
        return self.dropout(X)



class ATTENTION1(nn.Module):
    def __init__(self, num_states=4):
        super(ATTENTION1, self).__init__()
        num_heads=1
        hidden_dim=4
        self.num_states=4
        
        self.input_dim = num_states
        input_dim=num_states

        self.input_projection_1 = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.input_projection1_1 = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.input_projection2_1 = nn.Linear(hidden_dim, hidden_dim,bias=False)
        self.attention_1 = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)
        self.fc_1 = nn.Linear(hidden_dim ,hidden_dim)
        

    def forward(self,  Q, K,V):

        seq_len = Q.size(1)
        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=Q.device), diagonal=1)
        # print(attn_mask)
        attn_mask = attn_mask.masked_fill(attn_mask == 1, float('-inf')).masked_fill(attn_mask == 0, float(0.0))

        
        projected_inputs = self.input_projection_1(Q)  # [batch, seq_len, hidden_dim]
        projected_inputs1 = self.input_projection1_1(K)  # [batch, seq_len, hidden_dim]
        projected_inputs2 = self.input_projection2_1(V)  # [batch, seq_len, hidden_dim]
        attn_output, _ = self.attention_1(projected_inputs, projected_inputs1, projected_inputs2, attn_mask=attn_mask)  # [batch, seq_len, hidden_dim]
        pred = (F.leaky_relu(self.fc_1(attn_output))+ V )   # [batch, seq_len, output_dim]

        return pred



In [10]:

class Net(nn.Module):
    def __init__(self, num_states=4):
        super(Net, self).__init__()
        num_heads=1
        hidden_dim=4
        num_hiddens=4
        self.num_states=4
        self.s_0 = nn.Parameter(torch.rand(num_states), requires_grad=True)
        self.s_01 = nn.Parameter(torch.rand(num_states), requires_grad=True)
        self.mu = nn.Parameter(torch.rand(num_states), requires_grad=True)
        self.mu1 = nn.Parameter(torch.rand(num_states), requires_grad=True)
        self.sigma =nn.Parameter(torch.zeros(num_states), requires_grad=True)
        self.sigma1 = nn.Parameter(torch.zeros(num_states), requires_grad=True)

        self.base =3*(torch.tensor([0.0,0.25,0.5,1]))
        # self.dropout = nn.Dropout(dropout)
        # Create a long enough P
        max_len=300
        self.P = torch.zeros((1, max_len, num_hiddens))
        X = 3*torch.arange(max_len, dtype=torch.float32).reshape(
            -1, 1)/max_len #/ torch.pow(10000, torch.arange(
            #0, num_hiddens, 2, dtype=torch.float32) / num_hiddens)
        for i in range(4):
            self.P[:, :, i:i+1] = torch.cos(self.base[i]-X)

        
        self.input_dim = num_states
        input_dim=num_states
        
        self.fc = nn.Linear(input_dim+1 ,hidden_dim,bias=False)
        self.fc1 = nn.Linear(input_dim+1 ,hidden_dim,bias=False)
        self.fc2 = nn.Linear(1 ,hidden_dim)
        self.fc3 = nn.Linear(1 ,hidden_dim)
        self.fc4 = nn.Linear(hidden_dim ,hidden_dim)
        self.fc5 = nn.Linear(hidden_dim ,hidden_dim)
        self.fc6 =nn.Sequential(
    nn.Linear(4, 4),  # Input layer
    nn.LeakyReLU(),           # Activation function
    nn.Linear(4, 4),   # Hidden layer
    nn.LeakyReLU(),           # Activation function
    nn.Linear(4, 1)     # Output layer
)
        self.fc7 = nn.Sequential(
    nn.Linear(4, 4),  # Input layer
    nn.LeakyReLU(),           # Activation function
    nn.Linear(4, 4),   # Hidden layer
    nn.LeakyReLU(),           # Activation function
    nn.Linear(4, 1)     # Output layer
)
        
        self.A1=ATTENTION()
        self.A2=ATTENTION()
        self.A3=ATTENTION()
        self.A4=ATTENTION()
        self.A5=ATTENTION()
        self.A6=ATTENTION()
        self.A7=ATTENTION()
        self.A8=ATTENTION()
        self.A9=ATTENTION()
        self.A10=ATTENTION()

        # LSTM Encoder
        self.lstm1 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers=2, bidirectional=False)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers=2, bidirectional=False)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers=2, bidirectional=False)
        self.lstm4 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers=2, bidirectional=False)
        

        self.pos_encoding = PositionalEncoding(hidden_dim, 0)
    def forward(self,  t, pred,pred1, prob=0.3):

        s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(4,1,1)
        s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(4,1,1)

        # print(s_0.shape)
        exp_mu = torch.exp(self.mu)           # Step 1: Take exponential
        cumsum_mu = torch.cumsum(exp_mu, dim=0)


        exp_mu1 = torch.exp(self.mu1)           # Step 1: Take exponential
        cumsum_mu1 = torch.cumsum(exp_mu1, dim=0)

        
        pred_base=pred.clone()
        pred1_base=pred1.clone()

        pred=pred.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()
        pred1=pred1.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()

        new_pred = pred.clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
            
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred=new_pred.clone()
        
        new_pred = pred1.clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred1=new_pred.clone()

        # Generate a binary mask to decide which timestamps to replace
        mask = (torch.rand(4, pred.shape[1], 1, device=pred.device) < prob).float()
        replacement = torch.full_like(pred, -1.0)
        pred = pred * (1 - mask) + replacement * mask
        pred1 = pred1 * (1 - mask) + replacement * mask
        
        # print(pred.shape)
        pred=pred/torch.sum(pred,dim=2, keepdim=True)
        pred1=pred1/torch.sum(pred1,dim=2, keepdim=True)
        # print(pred[0,0])

        position = torch.arange(0, t, device=device).unsqueeze(0).repeat(4,1).unsqueeze(2).float()
        # print(position.shape)
        t1=F.tanh(self.fc2(position)).float()
        t2=F.tanh(self.fc3(position)).float()
        # t2=self.fc3(position).float()
        # pred=torch.concat([pred, position], dim=2)
        # pred1=torch.concat([pred1, position], dim=2)
        # print(pred.shape)
        


        # pred = F.leaky_relu(self.fc(pred))
        # pred1 = F.leaky_relu(self.fc1(pred1))
        pred_base1=pred.clone()
        pred1_base1=pred1.clone()

        pred_1, _ = self.lstm1(pred)
        pred1_1, _ = self.lstm2(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1

        
        # pred = F.leaky_relu(self.fc(pred)) + P
        # pred1 = F.leaky_relu(self.fc1(pred1)) + P
        # pred = pred + t1
        # pred1 = pred1 + t1
        


        
        pred=self.A1(pred,pred,pred,pred_base1)
        # pred=self.A2(pred,pred,pred,t1)
        # pred=self.A3(pred,pred,pred,t1)

        
        pred1=self.A4(pred1,pred1,pred1,pred1_base1)
        # pred1=self.A5(pred1,pred1,pred1,t1)
        # pred1=self.A6(pred1,pred1,pred1,t1)

        pred_1, _ = self.lstm3(pred)
        pred1_1, _ = self.lstm4(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1


        

        pred_1=self.A7(pred,pred1,pred,pred)
        # pred_1=self.A8(pred_1,pred1,pred_1)

        
        pred1_1=self.A9(pred1,pred ,pred1,pred1)
        # pred1_1=self.A10(pred1_1,pred ,pred1_1)

        pred=pred_1.clone()
        pred1=pred1_1.clone()


        pred = F.softmax(self.fc4(pred)+pred, dim=2)
        pred1 = F.softmax(self.fc5(pred1)+pred1, dim=2)

        # print(pred.shape)

        

        

        
        pred_1=torch.zeros((pred.shape[0], pred.shape[1],self.num_states))
        pred1_1=torch.zeros((pred1.shape[0], pred1.shape[1],self.num_states))
        pred=torch.concat([s_0,pred], dim=1)
        pred1=torch.concat([s_01,pred1], dim=1)
        # print(pred.shape)
        
        
        
        # print(pred.shape)


        
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
             
            pred_1[:,:,i] = o_1*pred[:,:-1,i] 


        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            pred1_1[:,:,i] = o_1*pred1[:,:-1,i] 


    
        KL=torch.sum(F.softmax(pred_1, dim=2)*torch.log(pred[0,:-1]/F.softmax(pred_1, dim=2)))
        KL+=torch.sum(F.softmax(pred1_1, dim=2)*torch.log(pred1[0,:-1]/F.softmax(pred1_1, dim=2)))
        
        pred_1=torch.sum(pred_1, dim=2)
        pred_1=torch.log(pred_1)
        pred_1=torch.sum(pred_1, dim=1)


        pred1_1=torch.sum(pred1_1, dim=2)
        pred1_1=torch.log(pred1_1)
        pred1_1=torch.sum(pred1_1, dim=1)


        


        return pred_1, pred1_1,KL



    

    def training1(self,  t, pred,pred1,m1,m2, prob=0.0):

        s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)
        s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)

        # print(s_0.shape)
        exp_mu = torch.exp(self.mu)           # Step 1: Take exponential
        cumsum_mu = torch.cumsum(exp_mu, dim=0)


        exp_mu1 = torch.exp(self.mu1)           # Step 1: Take exponential
        cumsum_mu1 = torch.cumsum(exp_mu1, dim=0)

        
        pred_base=pred.clone()
        pred1_base=pred1.clone()

        #pred=pred.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()
        #pred1=pred1.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()

        new_pred = pred.clone().unsqueeze(2).repeat(1,1,4).clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
            #print(o_1.shape)
            #print(new_pred[:,:,i].shape)
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred=new_pred.clone()
        
        new_pred = pred1.clone().unsqueeze(2).repeat(1,1,4).clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred1=new_pred.clone()
        if prob>0.0:
            mask = (torch.rand(100, pred.shape[1], 1, device=pred.device) < prob).float()
            shape = pred.shape[1]
            threshold = torch.randint(low=pred.shape[1]// 6 + 1, high=shape, size=(1,), device=pred.device).item()
            
            # Create a mask based on the threshold
            # Generate a range tensor of shape `pred.shape[1]`
            range_tensor = torch.arange(pred.shape[1], device=pred.device).unsqueeze(0).repeat(pred.shape[0], 1)
            mask = (range_tensor >= threshold).float().unsqueeze(-1)  # Add an extra dimension to match `pred`
            replacement = torch.full_like(pred, 1.0)
            pred = pred * (1 - mask) + replacement * mask
            pred1 = pred1 * (1 - mask) + replacement * mask


        
        # print(pred.shape)
        pred=pred/torch.sum(pred,dim=2, keepdim=True)
        pred1=pred1/torch.sum(pred1,dim=2, keepdim=True)
        # print(pred[0,0])


        # pred = F.leaky_relu(self.fc(pred))
        # pred1 = F.leaky_relu(self.fc1(pred1))
        pred_base1=pred.clone()
        pred1_base1=pred1.clone()

        pred_1, _ = self.lstm1(pred)
        pred1_1, _ = self.lstm2(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred=self.A1(pred,pred,pred,pred_base1)
        pred1=self.A4(pred1,pred1,pred1,pred1_base1)
        pred_1, _ = self.lstm3(pred)
        pred1_1, _ = self.lstm4(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred_1=self.A7(pred,pred1,pred,pred)
        pred1_1=self.A9(pred1,pred ,pred1,pred1)
        pred=pred_1.clone()
        pred1=pred1_1.clone()
        pred = F.softmax(self.fc4(pred)+pred, dim=2)
        pred1 = F.softmax(self.fc5(pred1)+pred1, dim=2)

        

        
        pred_1=torch.zeros((pred.shape[0], pred.shape[1],self.num_states)).cuda()
        pred1_1=torch.zeros((pred1.shape[0], pred1.shape[1],self.num_states)).cuda()
        pred=torch.concat([s_0,pred], dim=1)
        pred1=torch.concat([s_01,pred1], dim=1)
        # print(pred.shape)
        
        
        
        # print(pred.shape)


        
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
             
            pred_1[:,:,i] = o_1*pred[:,:-1,i] 


        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            pred1_1[:,:,i] = o_1*pred1[:,:-1,i] 


        RUL=self.fc6(pred_1)
        RUL1=self.fc7(pred1_1)
        KL=torch.sum(F.softmax(pred_1, dim=2)*torch.log(pred[0,:-1]/F.softmax(pred_1, dim=2)))
        KL+=torch.sum(F.softmax(pred1_1, dim=2)*torch.log(pred1[0,:-1]/F.softmax(pred1_1, dim=2)))
        
        pred_1=torch.sum(pred_1, dim=2)
        pred_1=torch.log(pred_1)
        pred_1=torch.sum(pred_1*m1, dim=1)


        pred1_1=torch.sum(pred1_1, dim=2)
        pred1_1=torch.log(pred1_1)
        pred1_1=torch.sum(pred1_1*m2, dim=1)


        


        return pred_1, pred1_1,KL,RUL,RUL1




    
    def state_filter(self,  t, pred,pred1,m=1,m1=1 ,prob=0.0):
        s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(4,1,1)
        s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(4,1,1)

        # print(s_0.shape)
        exp_mu = torch.exp(self.mu)           # Step 1: Take exponential
        cumsum_mu = torch.cumsum(exp_mu, dim=0)


        exp_mu1 = torch.exp(self.mu1)           # Step 1: Take exponential
        cumsum_mu1 = torch.cumsum(exp_mu1, dim=0)

        
        pred_base=pred.clone()
        pred1_base=pred1.clone()

        pred=pred.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()
        pred1=pred1.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()

        p1=torch.ones(pred.shape)
        p2=torch.ones(pred.shape)


        new_pred = pred.clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
            
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred=new_pred.clone()
        
        new_pred = pred1.clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred1=new_pred.clone()

        pred=pred*m1+(1-m1)*p1
        pred1=pred1*m1+(1-m1)*p2
       
        # print(pred.shape)
        pred=pred/torch.sum(pred,dim=2, keepdim=True)
        pred1=pred1/torch.sum(pred1,dim=2, keepdim=True)
        # print(pred[0,0])

        position = torch.arange(0, t).unsqueeze(0).repeat(4,1).unsqueeze(2).float()
        # print(position.shape)
        t1=F.tanh(self.fc2(position)).float()
        t2=F.tanh(self.fc3(position)).float()
        # t2=self.fc3(position).float()
        # pred=torch.concat([pred, position], dim=2)
        # pred1=torch.concat([pred1, position], dim=2)
        # print(pred.shape)
        


        pred_base1=pred.clone()
        pred1_base1=pred1.clone()

        pred_1, _ = self.lstm1(pred)
        pred1_1, _ = self.lstm2(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred=self.A1(pred,pred,pred,pred_base1)
        pred1=self.A4(pred1,pred1,pred1,pred1_base1)
        pred_1, _ = self.lstm3(pred)
        pred1_1, _ = self.lstm4(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred_1=self.A7(pred,pred1,pred,pred)
        pred1_1=self.A9(pred1,pred ,pred1,pred1)
        pred=pred_1.clone()
        pred1=pred1_1.clone()
        pred = F.softmax(self.fc4(pred)+pred, dim=2)
        pred1 = F.softmax(self.fc5(pred1)+pred1, dim=2)

        

        

        
        pred_1=torch.zeros((pred.shape[0], pred.shape[1],self.num_states))
        pred1_1=torch.zeros((pred1.shape[0], pred1.shape[1],self.num_states))
        pred=torch.concat([s_0,pred], dim=1)
        pred1=torch.concat([s_01,pred1], dim=1)


        

        
        
        
        
        
        
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
             
            pred_1[:,:,i] = o_1*pred[:,:-1,i] 


        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            pred1_1[:,:,i] = o_1*pred1[:,:-1,i] 


        pred_1=torch.softmax(pred_1, dim=2)
        # pred_1=torch.log(pred_1)
        # pred_1=torch.sum(pred_1, dim=1)


        pred1_1=torch.softmax(pred1_1, dim=2)
        # pred1_1=torch.log(pred1_1)
        # pred1_1=torch.sum(pred1_1, dim=1)


        return pred_1, pred1_1
    def state_filter1(self,  t, pred,pred1,m=1,m1=1 ,prob=0.0):
        s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)
        s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)

        # print(s_0.shape)
        exp_mu = torch.exp(self.mu)           # Step 1: Take exponential
        cumsum_mu = torch.cumsum(exp_mu, dim=0)


        exp_mu1 = torch.exp(self.mu1)           # Step 1: Take exponential
        cumsum_mu1 = torch.cumsum(exp_mu1, dim=0)

        
        pred_base=pred.clone()
        pred1_base=pred1.clone()

        #pred=pred.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()
        #pred1=pred1.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()

        new_pred = pred.clone().unsqueeze(2).repeat(1,1,4).clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
            #print(o_1.shape)
            #print(new_pred[:,:,i].shape)
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred=new_pred.clone()
        
        new_pred = pred1.clone().unsqueeze(2).repeat(1,1,4).clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred1=new_pred.clone()
        if prob>0.0:
            mask = (torch.rand(100, pred.shape[1], 1, device=pred.device) < prob).float()
            shape = pred.shape[1]
            threshold = torch.randint(low=pred.shape[1]// 6 + 1, high=shape, size=(1,), device=pred.device).item()
            
            # Create a mask based on the threshold
            # Generate a range tensor of shape `pred.shape[1]`
            range_tensor = torch.arange(pred.shape[1], device=pred.device).unsqueeze(0).repeat(pred.shape[0], 1)
            mask = (range_tensor >= threshold).float().unsqueeze(-1)  # Add an extra dimension to match `pred`
            replacement = torch.full_like(pred, 1.0)
            pred = pred * (1 - mask) + replacement * mask
            pred1 = pred1 * (1 - mask) + replacement * mask


        
        # print(pred.shape)
        pred=pred/torch.sum(pred,dim=2, keepdim=True)
        pred1=pred1/torch.sum(pred1,dim=2, keepdim=True)
        # print(pred[0,0])


        # pred = F.leaky_relu(self.fc(pred))
        # pred1 = F.leaky_relu(self.fc1(pred1))
        pred_base1=pred.clone()
        pred1_base1=pred1.clone()

        pred_1, _ = self.lstm1(pred)
        pred1_1, _ = self.lstm2(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred=self.A1(pred,pred,pred,pred_base1)
        pred1=self.A4(pred1,pred1,pred1,pred1_base1)
        pred_1, _ = self.lstm3(pred)
        pred1_1, _ = self.lstm4(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred_1=self.A7(pred,pred1,pred,pred)
        pred1_1=self.A9(pred1,pred ,pred1,pred1)
        pred=pred_1.clone()
        pred1=pred1_1.clone()
        pred = F.softmax(self.fc4(pred)+pred, dim=2)
        pred1 = F.softmax(self.fc5(pred1)+pred1, dim=2)
        

        

        
        pred_1=torch.zeros((pred.shape[0], pred.shape[1],self.num_states)).cuda()
        pred1_1=torch.zeros((pred1.shape[0], pred1.shape[1],self.num_states)).cuda()
        pred=torch.concat([s_0,pred], dim=1)
        pred1=torch.concat([s_01,pred1], dim=1)
        # print(pred.shape)
        
        
        
        # print(pred.shape)


        
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
             
            pred_1[:,:,i] = o_1*pred[:,:-1,i] 


        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            pred1_1[:,:,i] = o_1*pred1[:,:-1,i] 


    
        KL=torch.sum(F.softmax(pred_1, dim=2)*torch.log(pred[0,:-1]/F.softmax(pred_1, dim=2)))
        KL+=torch.sum(F.softmax(pred1_1, dim=2)*torch.log(pred1[0,:-1]/F.softmax(pred1_1, dim=2)))
        
        pred_1=torch.softmax(pred_1, dim=2)
        # pred_1=torch.log(pred_1)
        # pred_1=torch.sum(pred_1, dim=1)


        pred1_1=torch.softmax(pred1_1, dim=2)
        # pred1_1=torch.log(pred1_1)
        # pred1_1=torch.sum(pred1_1, dim=1)



        return pred_1, pred1_1
        
    def new_sample(self,  t, pred,pred1,m=1,m1=1 ,prob=0.0):
        s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(4,1,1)
        s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(4,1,1)

        # print(s_0.shape)
        exp_mu = torch.exp(self.mu)           # Step 1: Take exponential
        cumsum_mu = torch.cumsum(exp_mu, dim=0)


        exp_mu1 = torch.exp(self.mu1)           # Step 1: Take exponential
        cumsum_mu1 = torch.cumsum(exp_mu1, dim=0)

        
        pred_base=pred.clone()
        pred1_base=pred1.clone()

        pred=pred.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()
        pred1=pred1.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()

        p1=torch.ones(pred.shape)*1.0
        p2=torch.ones(pred.shape)*1.0


        new_pred = pred.clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
            
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred=new_pred.clone()
        
        new_pred = pred1.clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred1=new_pred.clone()
        
        #print(m1.repeat(1,1).unsqueeze(2).repeat(4,1,4).shape)
        #pred=pred*m1.repeat(1,1).unsqueeze(2).repeat(4,1,4)+(~m1.repeat(1,1).unsqueeze(2).repeat(4,1,4))*p1
        #pred1=pred1*m1.repeat(1,1).unsqueeze(2).repeat(4,1,4)+(~m1.repeat(1,1).unsqueeze(2).repeat(4,1,4))*p2
       
        # print(pred.shape)
        pred=pred/torch.sum(pred,dim=2, keepdim=True)*1.0
        pred1=pred1/torch.sum(pred1,dim=2, keepdim=True)*1.0
        # print(pred[0,0])

        position = torch.arange(0, t).unsqueeze(0).repeat(4,1).unsqueeze(2).float()
        # print(position.shape)
        t1=F.tanh(self.fc2(position)).float()
        t2=F.tanh(self.fc3(position)).float()
        # t2=self.fc3(position).float()
        # pred=torch.concat([pred, position], dim=2)
        # pred1=torch.concat([pred1, position], dim=2)
        # print(pred.shape)
        


        pred_base1=pred.clone()*1.0
        pred1_base1=pred1.clone()*1.0

        pred_1, _ = self.lstm1(pred)
        pred1_1, _ = self.lstm2(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred=self.A1(pred,pred,pred,pred_base1)
        pred1=self.A4(pred1,pred1,pred1,pred1_base1)
        pred_1, _ = self.lstm3(pred)
        pred1_1, _ = self.lstm4(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred_1=self.A7(pred,pred1,pred,pred)
        pred1_1=self.A9(pred1,pred ,pred1,pred1)
        pred=pred_1.clone()
        pred1=pred1_1.clone()
        pred = F.softmax(self.fc4(pred)+pred, dim=2)
        pred1 = F.softmax(self.fc5(pred1)+pred1, dim=2)

        

        

        
        pred_1=torch.zeros((pred.shape[0], pred.shape[1],self.num_states))
        pred1_1=torch.zeros((pred1.shape[0], pred1.shape[1],self.num_states))
        pred=torch.concat([s_0,pred], dim=1)[:,-1]
        pred1=torch.concat([s_01,pred1], dim=1)[:,-1]

        o = Normal(torch.sum(cumsum_mu*pred,dim=1), torch.sum(torch.exp(self.sigma)**2*pred,dim=1)**0.5).sample()
        o1 = Normal(torch.sum(cumsum_mu1*pred1,dim=1), torch.sum(torch.exp(self.sigma1)**2*pred1,dim=1)**0.5).sample()

        


        

        
        
        
        
        
    
       
        return o[0], o1[0]

    def new_sample1(self,  t, pred,pred1,m=1,m1=1 ,prob=0.0):
        s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)
        s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)

        # print(s_0.shape)
        exp_mu = torch.exp(self.mu)           # Step 1: Take exponential
        cumsum_mu = torch.cumsum(exp_mu, dim=0)


        exp_mu1 = torch.exp(self.mu1)           # Step 1: Take exponential
        cumsum_mu1 = torch.cumsum(exp_mu1, dim=0)

        
        pred_base=pred.clone()
        pred1_base=pred1.clone()

        #pred=pred.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()
        #pred1=pred1.unsqueeze(0).repeat(1,1).unsqueeze(2).repeat(4,1,4).clone()

        new_pred = pred.clone().unsqueeze(2).repeat(1,1,4).clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu[i], torch.exp(self.sigma[i]))
            o_1 = torch.exp(dist.log_prob(pred_base))
            #print(o_1.shape)
            #print(new_pred[:,:,i].shape)
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred=new_pred.clone()
        
        new_pred = pred1.clone().unsqueeze(2).repeat(1,1,4).clone()
        for i in range(self.num_states):
            dist = Normal(cumsum_mu1[i], torch.exp(self.sigma1[i]))
            o_1 = torch.exp(dist.log_prob(pred1_base))
             
            new_pred[:,:,i] = o_1#*1/self.num_states
        pred1=new_pred.clone()
        if prob>0.0:
            mask = (torch.rand(100, pred.shape[1], 1, device=pred.device) < prob).float()
            shape = pred.shape[1]
            threshold = torch.randint(low=pred.shape[1]// 6 + 1, high=shape, size=(1,), device=pred.device).item()
            
            # Create a mask based on the threshold
            # Generate a range tensor of shape `pred.shape[1]`
            range_tensor = torch.arange(pred.shape[1], device=pred.device).unsqueeze(0).repeat(pred.shape[0], 1)
            mask = (range_tensor >= threshold).float().unsqueeze(-1)  # Add an extra dimension to match `pred`
            replacement = torch.full_like(pred, 1.0)
            pred = pred * (1 - mask) + replacement * mask
            pred1 = pred1 * (1 - mask) + replacement * mask


        
        # print(pred.shape)
        pred=pred/torch.sum(pred,dim=2, keepdim=True)
        pred1=pred1/torch.sum(pred1,dim=2, keepdim=True)
        # print(pred[0,0])


        # pred = F.leaky_relu(self.fc(pred))
        # pred1 = F.leaky_relu(self.fc1(pred1))
        pred_base1=pred.clone()
        pred1_base1=pred1.clone()

        pred_1, _ = self.lstm1(pred)
        pred1_1, _ = self.lstm2(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred=self.A1(pred,pred,pred,pred_base1)
        pred1=self.A4(pred1,pred1,pred1,pred1_base1)
        pred_1, _ = self.lstm3(pred)
        pred1_1, _ = self.lstm4(pred1)
        pred=F.tanh(pred)+pred_1
        pred1=F.tanh(pred1)+pred1_1
        pred_1=self.A7(pred,pred1,pred,pred)
        pred1_1=self.A9(pred1,pred ,pred1,pred1)
        pred=pred_1.clone()
        pred1=pred1_1.clone()
        pred = F.softmax(self.fc4(pred)+pred, dim=2)
        pred1 = F.softmax(self.fc5(pred1)+pred1, dim=2)
        

        

        
        pred_1=torch.zeros((pred.shape[0], pred.shape[1],self.num_states)).cuda()
        pred1_1=torch.zeros((pred1.shape[0], pred1.shape[1],self.num_states)).cuda()
        pred=torch.concat([s_0,pred], dim=1)[:,-1]
        pred1=torch.concat([s_01,pred1], dim=1)[:,-1]

        B,  N = pred.shape 
        pred=torch.multinomial(pred.view(-1, N) , num_samples=1).view(B)

        B,  N = pred1.shape 
        pred1=torch.multinomial(pred1.view(-1, N) , num_samples=1).view(B)
        

        o = Normal(cumsum_mu1[pred], torch.exp(self.sigma1[pred])).sample()
        o1 = Normal(cumsum_mu1[pred1], torch.exp(self.sigma1[pred1])).sample()

        


        

        
        
        
        
        
    
       
        return o, o1





In [11]:
import torch

def pad_sequences(sequences, padding_value=0):
    """
    Pads a list of variable-length sequences with a padding value.
    """
    max_length = max(len(seq) for seq in sequences)
    padded_sequences = torch.full((len(sequences), max_length), padding_value, dtype=torch.float32)
    mask = torch.zeros((len(sequences), max_length), dtype=torch.bool)
    
    for i, seq in enumerate(sequences):
        length = len(seq)
        padded_sequences[i, :length] = torch.tensor(seq, dtype=torch.float32)
        mask[i, :length] = True
    
    return padded_sequences, mask



In [12]:
def pad_sequences(sequences, padding_value=0):
    """
    Pads a list of variable-length sequences with the last value of each sequence.
    Returns the padded sequences and a mask indicating the original positions.
    """
    max_length = max(len(seq) for seq in sequences)
    padded_sequences = torch.zeros((len(sequences), max_length), dtype=torch.float32)
    mask = torch.zeros((len(sequences), max_length), dtype=torch.bool)
    
    for i, seq in enumerate(sequences):
        length = len(seq)
        padded_sequences[i, :length] = torch.tensor(seq, dtype=torch.float32)
        mask[i, :length] = True
        
        # Fill the remaining positions with the last value of the sequence
        if length > 0:  # Check to ensure sequence is not empty
            padded_sequences[i, length:] = seq[-1]
    
    return padded_sequences, mask

In [13]:
def pad_sequences(sequences, padding_value=0):
    """
    Pads a list of variable-length sequences with the last value of each sequence.
    Returns the padded sequences and a mask indicating the original positions.
    """
    max_length = max(len(seq) for seq in sequences)
    padded_sequences = torch.zeros((len(sequences), max_length), dtype=torch.float32)
    mask = torch.zeros((len(sequences), max_length), dtype=torch.bool)
    # Collect the last values from all sequences
    last_values = [seq[-1] for seq in sequences if len(seq) > 0]
    last_values_tensor = torch.tensor(last_values, dtype=torch.float32)
    for i, seq in enumerate(sequences):
        length = len(seq)
        padded_sequences[i, :length] = torch.tensor(seq, dtype=torch.float32)
        mask[i, :length] = True
        
        if length > 0:  # Ensure the sequence is not empty
            random_indices = torch.randint(0, len(last_values_tensor), (max_length - length,))
            random_values = last_values_tensor[random_indices]
            padded_sequences[i, length:] = random_values
    
    return padded_sequences, mask

In [14]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])

def var_diff(o_1):
    return torch.sum(o_1[1])

def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k


def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=(e2-e1)**2
    return k


In [15]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import random
import numpy as np
import time

In [16]:

# Load the dataset
pred1 = torch.load('TRAIN_DATA_NON_LINEAR.pth')

# Parameters for experiment
batch_size = 100
num_epochs = 10000
n_trials = 10
learning_rate = 0.01
device='cuda'
# Initialize result containers
all_losses = []
all_l1 = []
all_l2 = []
trial_times = []
final_mu_values = []

for trial in range(1):
    print(f"Trial {trial + 1}/{n_trials}")
    trial_start_time = time.time()

    net = Net().to(device)  # Initialize model
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.01, total_iters=num_epochs)

    net.train()
    trial_losses = []
    trial_l1 = []
    trial_l2 = []
    # Extract observations and underlying states
    observations_o = [o for o, _, _ in pred1]
    observations_o1 = [o for _, o, _ in pred1]
    times = torch.tensor([o for _, _, o in pred1]).cuda()
    true_states = []
    true_states1 = []
    observations_o, m1 = pad_sequences(observations_o, padding_value=-1)
    observations_o1, m2 = pad_sequences(observations_o1, padding_value=-1)
    observations_o=observations_o.cuda()
    observations_o1=observations_o1.cuda()
    m1=m1.cuda()
    m2=m2.cuda()
    m_1=m1.clone()
    m_2=m2.clone()
    print(m1.sum())
    print(m1.shape)
    #m_1[:,:-200]=1
    #m_2[:,:-200]=1
    print(m_1.sum())
    print(m_1.shape)
    

    for epoch in range(num_epochs):
        net.t = 0  # Reset any time-dependent states

        # Create random batch indices
    

        loss = 0.0
        
        optimizer.zero_grad()
        l1,l2,_,r1,r2=net.training1(0, observations_o, observations_o1,m_1,m_2, 0.0)
        lengths=times.unsqueeze(1).repeat(1,observations_o.shape[1]).unsqueeze(2).repeat(1,1,1)-torch.arange(0,observations_o.shape[1]).cuda().unsqueeze(0).repeat(100,1).unsqueeze(2).repeat(1,1,1)
        mse=(r1-lengths)**2*m1.unsqueeze(2).repeat(1,1,1)+(r2-lengths)**2*m2.unsqueeze(2).repeat(1,1,1)
        loss -= (torch.mean(l1) + torch.mean(l2))-torch.mean(mse) #- 0.01*torch.mean(torch.log(torch.exp(net.sigma1)))

        # Add L1 regularization
        # l1_reg = torch.tensor(0., device=device)
        # for param in net.parameters():
        #     l1_reg += torch.norm(param, 1)

        # # Update loss
        # loss += 0.01 * l1_reg
        #print(loss)
        trial_losses.append(loss.item())

        # Compute gradients and update parameters
        loss.backward()
        optimizer.step()
        scheduler.step()
    l1,l2,_,r1,r2=net.training1(0, observations_o, observations_o1,m1,m2, 0.0)
    loss = (torch.mean(l1) + torch.mean(l2))
    # Save trial results
    print((loss.item()))
    print(net.mu.grad)
    print(torch.cumsum(torch.exp(net.mu),dim=0))
    all_losses.append(loss.item())
    trial_times.append(time.time() - trial_start_time)
    final_mu_values.append(torch.cumsum(torch.exp(net.mu),dim=0).detach().cpu().numpy())


# Compute statistics across trials
mean_losses = np.mean(all_losses, axis=0)
std_losses = np.std(all_losses, axis=0)
mean_trial_time = np.mean(trial_times)
std_trial_time = np.std(trial_times)

# # Plot loss curve
# plt.figure(figsize=(10, 6))
# for trial_losses in all_losses:
#     plt.plot(trial_losses, alpha=0.7)
# plt.title('Loss Curve Across Trials')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.grid(True)
# plt.show()



Trial 1/10


C:\Users\Administrator\AppData\Local\Temp\ipykernel_27464\1215635058.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pred1 = torch.load('TRAIN_DATA_NON_LINEAR.pth')
C:\U

tensor(20630, device='cuda:0')
torch.Size([100, 362])
tensor(20630, device='cuda:0')
torch.Size([100, 362])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_27464\3230021381.py:237: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)

KeyboardInterrupt



In [17]:
final_mu_values



[]

In [18]:
torch.cumsum(torch.exp(net.mu1),dim=0).detach().cpu().numpy()

array([0.47946134, 0.9046289 , 1.4119973 , 3.2913659 ], dtype=float32)

In [19]:
torch.exp(net.sigma1)

tensor([0.3844, 0.4094, 0.7351, 1.1682], device='cuda:0',
       grad_fn=<ExpBackward0>)

In [20]:
torch.exp(net.sigma)

tensor([0.0798, 0.0626, 0.0828, 0.1762], device='cuda:0',
       grad_fn=<ExpBackward0>)

In [21]:
torch.cumsum(torch.exp(net.mu),dim=0).detach().cpu().numpy()


array([0.08480553, 0.23566172, 0.2584929 , 0.3078511 ], dtype=float32)

In [22]:
pred1 = torch.load('TEST_DATA_NON_LINEAR.pth')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_27464\1679614673.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pred1 = torch.load('TEST_DATA_NON_LINEAR.pth')


In [36]:
observations_o = [o for o, _, _ in pred1]
observations_o1 = [o for _,o, _ in pred1]
t = [o for _, _, o in pred1]
true_states = []
true_states1 = []
observations_o, m1 = pad_sequences(observations_o, padding_value=-1)
observations_o1, m2 = pad_sequences(observations_o1, padding_value=-1)
observations_o=observations_o.cuda()
observations_o1=observations_o1.cuda()
observations_o=observations_o.cuda()
observations_o1=observations_o1.cuda()
m1=m1.cuda()
m2=m2.cuda()
l1,l2,_,r1,r2=net.training1(0, observations_o, observations_o1,m1,m2, 0.0)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_27464\2254581785.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_sequences[i, :length] = torch.tensor(seq, dtype=torch.float32)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_27464\3230021381.py:236: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s_0=F.softmax(self.s_0).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_27464\3230021381.py:237: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s_01=F.softmax(self.s_01).unsqueeze(0).repeat(1,1).unsqueeze(1).repeat(100,1,1)


In [37]:
observations_o.shape

torch.Size([100, 303])

In [90]:
predictions=torch.tensor([r2[i,t[i]-1 ].mean().item() for i in range(100)])
predictions

tensor([184.1531,  63.0268,  76.6661, 123.3271,  73.7334,  53.6943, 169.3956,
        134.8301,  98.8976,  66.0604,  63.5953,  83.0892,  98.0082,  18.1069,
        106.8268, 136.6696, 109.9465,  23.7119, 139.3002,  96.8478,  59.6386,
         75.9416, 115.9222,  67.9871,  40.2340,  11.0272,  69.1179, 119.4427,
         18.0719,  17.2512,  40.2249,  31.4522,  78.0246, 191.9198,  60.0496,
         20.3063,  21.9158,  16.5576,  98.9570, 184.3501,  45.4114,  30.3729,
         52.5919,  71.2320,  14.6599, 136.3511,  83.6561,  93.4339,  45.5760,
         29.3230, 132.0515,  96.7566,  29.9210, 132.1112,  30.6914, 134.5907,
        144.4327, 127.4851,  20.6146,  45.9540,  47.4967,  26.4309, 144.8436,
         22.7790, 100.2603,  15.4701, 101.3625, 168.5149,  61.0924, 134.0115,
         36.4732, 144.1476,  62.8670,  74.0687,  15.4671,  37.7099, 175.7345,
         65.6912,  65.3018, 143.2440,  17.4550,  16.5584, 104.3225,  97.1394,
        132.7302,  73.4243, 128.9994, 148.2531, 163.8143, 109.29

In [91]:
# Load the data
import pandas as pd
df = pd.read_csv('../CMAPSSData/RUL_FD001.txt', sep=" ",names=['t']).index.values

In [92]:
true=torch.tensor(df)
true

tensor([112,  98,  69,  82,  91,  93,  91,  95, 111,  96,  97, 124,  95, 107,
         83,  84,  50,  28,  87,  16,  57, 111, 113,  20, 145, 119,  66,  97,
         90, 115,   8,  48, 106,   7,  11,  19,  21,  50, 142,  28,  18,  10,
         59, 109, 114,  47, 135,  92,  21,  79, 114,  29,  26,  97, 137,  15,
        103,  37, 114, 100,  21,  54,  72,  28, 128,  14,  77,   8, 121,  94,
        118,  50, 131, 126, 113,  10,  34, 107,  63,  90,   8,   9, 137,  58,
        118,  89, 116, 115, 136,  28,  38,  20,  85,  55, 128, 137,  82,  59,
        117,  20])

In [98]:
torch.mean((predictions[true>70]-true[true>70])**2)**0.5

tensor(56.3022)

In [99]:
predictions-true

tensor([  72.1531,  -34.9732,    7.6661,   41.3271,  -17.2666,  -39.3057,
          78.3956,   39.8301,  -12.1024,  -29.9396,  -33.4047,  -40.9108,
           3.0082,  -88.8931,   23.8268,   52.6696,   59.9465,   -4.2881,
          52.3002,   80.8478,    2.6386,  -35.0584,    2.9222,   47.9871,
        -104.7660, -107.9728,    3.1179,   22.4427,  -71.9281,  -97.7488,
          32.2249,  -16.5478,  -27.9754,  184.9198,   49.0496,    1.3063,
           0.9158,  -33.4424,  -43.0430,  156.3501,   27.4114,   20.3729,
          -6.4081,  -37.7680,  -99.3401,   89.3511,  -51.3439,    1.4339,
          24.5760,  -49.6770,   18.0515,   67.7566,    3.9210,   35.1112,
        -106.3086,  119.5907,   41.4327,   90.4851,  -93.3854,  -54.0460,
          26.4967,  -27.5691,   72.8436,   -5.2210,  -27.7397,    1.4701,
          24.3625,  160.5149,  -59.9076,   40.0115,  -81.5268,   94.1476,
         -68.1330,  -51.9313,  -97.5329,   27.7099,  141.7345,  -41.3088,
           2.3018,   53.2440,    9.455

In [100]:
torch.mean((predictions-true)**2)**0.5


tensor(62.9702)

In [ ]:
all_l1=[]
all_l2=[]
for j in [450]*1:
    print(j)
    net.cuda()
    observations_o = [o for o, _, _ in pred1]
    observations_o1 = [o for _, o, _ in pred1]
    true_states = []
    true_states1 = []   
    trial_l1=[]
    trial_l2=[]
    for i in range(0,100):
        o, o1, t = pred1[i]
        l=o.shape[0]
        
        o_1=o.float().clone().unsqueeze(0).repeat(100,1).cuda()
        o1_1=o1.float().clone().unsqueeze(0).repeat(100,1).cuda()
        for k in range(l,j):
            
            filtered_l1, filtered_l2 = net.new_sample1(t, o_1, o1_1)
        
            o_1=torch.concat([o_1,filtered_l1.reshape(100,1)],dim=1)
           
            o1_1=torch.concat([o1_1,filtered_l2.reshape(100,1)],dim=1)
            
        #filtered_l1, filtered_l2 = net.new_sample1(t, o_1, o1_1)
        filtered_l1, filtered_l2 = net.state_filter1(t, o_1, o1_1)
        trial_l1.append(torch.argmax(filtered_l1, dim=2).detach().cpu())
        trial_l2.append(torch.argmax(filtered_l2, dim=2).detach().cpu())
        
        

In [ ]:
all_l1=torch.stack(trial_l1)
all_l2=torch.stack(trial_l1)

In [ ]:
all_l1[3][0][observations_o[3].shape[0]:] 

In [ ]:
filtered_l1, filtered_l2 = net.state_filter1(t, o_1, o1_1)

In [ ]:
(torch.where(trial_l2[1] == 0))

In [ ]:
main_l=[]
for trial in range(100):
    r=[]
    mse_l1 = 0.0
    mse_l2 = 0.0
    total_states = 0

    l1 = all_l1[:,trial]
    l2 = all_l2[:,trial]
    print(l1.shape)
    for i in range(100):
        
        try:
            r.append((torch.where(l2[i][observations_o[i].shape[0]:] >=3)[0][0]).item())
            print("hi")
            
        except:
            try:
                r.append((torch.where(l2[i][observations_o[i].shape[0]:]==2)[0][0]).item())
                
            except:
                try:
                    r.append((torch.where(l2[i][observations_o[i].shape[0]:] ==1)[0][0]).item())
                    
                except:
                        r.append(420-observations_o[i].shape[0])
                        
            
    main_l.append(r)

In [ ]:
main_l=torch.tensor(main_l)

In [ ]:
main_l.shape

In [ ]:
predictions=torch.tensor(main_l).float().mean(axis=0).clip(0,1000)


In [ ]:
# Load the data
import pandas as pd
df = pd.read_csv('../CMAPSSData/RUL_FD001.txt', sep=" ",names=['t']).index.values

In [ ]:
true=torch.tensor(df)

In [ ]:
torch.save(main_l, 'prediction.pth')

In [ ]:
torch.round(predictions[true>70])

In [ ]:
true[true>70]

In [ ]:
torch.mean((predictions[true>70]-true[true>70])**2)**0.5

In [ ]:
average_state=torch.round(all_l2.float().mean(axis=1))

In [ ]:
torch.mean((predictions-true)**2)**0.5

In [ ]:
true

In [ ]:
(torch.tensor([o*1.0 for _, _, o in pred1]))

In [ ]:
torch.where(torch.argmax(trial_l2[0], dim=2)[0] == 0)[0]


In [ ]:
trial_l2[0][0]

In [ ]:
observations_o[0].shape

In [ ]:
trial_l2[0].shape

In [ ]:
torch.argmax(trial_l2[0], dim=2)[0]

In [ ]:
filtered_l1[0]

In [ ]:
o.shape

In [ ]:
plt.plot(trial_losses)
plt.show()

In [ ]:
np.min(trial_losses)

In [ ]:
all_losses

In [ ]:
# Compute statistics across trials
mean_losses = np.mean(all_losses, axis=0)
std_losses = np.std(all_losses, axis=0)
mean_trial_time = np.mean(trial_times)
std_trial_time = np.std(trial_times)

In [ ]:
# Output results
print("Training Results:")
print(f"Mean Loss (Final Epoch): {-mean_losses}, Std Loss: {std_losses}")
print(f"Mean Training Time: {mean_trial_time}s, Std Training Time: {std_trial_time}s")
print(f"Final mu values (across trials): {np.array(final_mu_values)}")

In [ ]:
pred1[0][3].shape

In [ ]:
# State filtering MSE
true_states = [t.numpy()[:, 0:1] for _, _, _, t in pred1]
true_states1 = [t.numpy()[:, 1:2] for _, _, _, t in pred1]

trial_mse_l1 = []
trial_mse_l2 = []

for trial in range(3):
    mse_l1 = 0.0
    mse_l2 = 0.0
    total_states = 0

    l1 = all_l1[trial]
    l2 = all_l2[trial]

    for filtered_l1, filtered_l2, ts, ts1 in zip(l1, l2, true_states, true_states1):
        mse_l1 += (np.diag(filtered_l1.detach().numpy() == ts) ).sum()
        # print(filtered_l1.shape)
        # print(ts.shape)
        mse_l2 += (np.diag(filtered_l2.detach().numpy() ==  ts1)).sum()
        total_states += ts.shape[0]
    # print(mse_l1)
    # print(total_states)
    

    mse_l1 /= total_states
    mse_l2 /= total_states

    trial_mse_l1.append(mse_l1)
    trial_mse_l2.append(mse_l2)

# Calculate average error and standard deviation
average_mse_l1 = np.mean(trial_mse_l1)
std_mse_l1 = np.std(trial_mse_l1)

average_mse_l2 = np.mean(trial_mse_l2)
std_mse_l2 = np.std(trial_mse_l2)

# Print results
print("State Filtering MSE Results:")
print(f"Average MSE for l1: {average_mse_l1:.4f}, Std Dev: {std_mse_l1:.4f}")
print(f"Average MSE for l2: {average_mse_l2:.4f}, Std Dev: {std_mse_l2:.4f}")

# Print MSE for each trial
for trial, (mse_l1, mse_l2) in enumerate(zip(trial_mse_l1, trial_mse_l2), 1):
    print(f"Trial {trial}: MSE l1 = {mse_l1:.4f}, MSE l2 = {mse_l2:.4f}")


In [ ]:
trial_mse_l2

In [ ]:
trial_mse_l1

In [ ]:
torch.tensor(true_states1[0]).reshape(-1)

In [ ]:
all_l1[3][-2]

In [ ]:
plt.plot(observations_o1[-2])
plt.show()

In [ ]:
torch.tensor(ts).reshape(-1)

In [ ]:
true_states1[-1]